# Music Similarity Annotation

Prerequisites:
- a running internet connection
- the python packages: pandas, numpy, ast, [pigeonXT](https://github.com/dennisbakhuis/pigeonXT) and IPython
- some interest in music ;)

Please use this notebook for music similarity annotation of retrieved versions from [YouTube](https://www.youtube.com/). Our goal is to annotate as many pairs of YouTube videos as possible. **A** should represent a work or song, while **B** represents a candidate video which we crawled from YouTube. We want to find out whether **B** is really a musical version of **A**.


Please annotate by conducting the following steps:
1. Set the variable `ANNOTATOR` in the code cell below to your initials. 
1. Execute the code cell below, this will show you a prompt with two embedded audios from YouTube
2. Listen to audio **A** by clicking on it. You do not need to listen to the full recording, but should try to figure out which song it is (if you know it) or try to remember some important part(s) about it (e.g. chorus). Some tips to simplify this step:
  - you can use the arrow buttons (`←`, `→`) to go backward and forward by 10 seconds and to adjust the volume. (`↑`, `↓`)
  - you can use `SPACE` to play and stop
  - sometimes the intro is sufficient to recognize a song, please still try to ensure that the audio is what you think it is by playing some other sections
3. Listen to **B** and do the same like you did for **A**. Try to figure out whether **B** is a musical version of **A**.
4. Select based on your decision:
    - `b_matches_a`: **B** is the piece as **A** (e.g same studio recording but YouTube uploader and quality might be different)
    - `b_version_a`: **B** is in fact a musical version of **A** but has some major changes (e.g. tempo, genre, instrumentation, changes in structure, harmony, melody)
    - `b_unrelated_a`: **B** is originating from a different song than **A**.
    - `b_nonmusic`: **B** does not contain music but noise only (e.g. speech, nature sounds, traffic sounds). 
    - `unsure`: You are uncertain about the relationship between the two recordings. You can use this option also in case you notice a bug (e.g. **A** is not a musical piece or one of the recordings is not available on YouTube)
5. The file `annotated.csv` is now updated. You now see the next recording and can do the same again. 
6. Once you want to stop annotating, save your `annotated.csv` file somewhere.
    
You can find some examples of relationships below the code cell.
    

The code in this notebook is based on [pigeonXT](https://github.com/dennisbakhuis/pigeonXT).



In [ ]:
df

In [ ]:
!pip install pigeonXT-jupyter
import pandas as pd
import numpy as np
import ast

from datetime import datetime
from pigeonXT import annotate
from IPython.display import display, YouTubeVideo

    
# annotator, please set to your initials
ANNOTATOR = input()

# full list
df = pd.read_csv('https://raw.githubusercontent.com/progsi/yt_music_annotation/main/eval_input.csv')
    
# previously annotated
try:
    df_annotated = pd.read_csv('annotated.csv', sep=';').dropna(subset=['label'])
except FileNotFoundError:
    df_annotated = pd.DataFrame([], columns=['example', 'changed', 'label', 'with_video', 'annotator', 'timestamp'])

# to tuple list
annotated_pairs = [ast.literal_eval(id_pair_annotated) for id_pair_annotated in df_annotated['example']]

# filter out previously annotated
id_pairs = [id_pair for id_pair in list(zip(df.query_id, df.candidate_id)) if id_pair not in annotated_pairs]
labels = ['b_version_a', 'b_matches_a', 'b_unrelated_a', 'b_nonmusic', 'unsure']


def annotateYouTubeAudio(indf, labels):

    def updateAnnotated(example, selected_label):
        global df_annotated
        df_annotated = pd.concat(
            [
                df_annotated, 
             pd.DataFrame(
                 {'example': [example], 'changed': [True], 'label': [selected_label], 'with_video': True, 'annotator': [ANNOTATOR], 'timestamp': datetime.now()})])
        df_annotated.reset_index(drop=True).to_csv('annotated.csv', sep=';', index=False)
        
    def show(i, example):
        if i == 0:
            print('A')
        elif i == 1:
            print('B')
        display(
            YouTubeVideo(
                example,
                showinfo=0,
                disablekb=0,
                modestbranding=1,
                rel=0,
                height=90,
                width=90     
            )
        )

    annotated = annotate( 
        id_pairs, 
        options=labels, 
        task_type='classification',
        shuffle=True,
        buttons_in_a_row=3,
        reset_buttons_after_click=True,
        include_next=False,
        display_fn=lambda filenames: [show(i, filename)  for i, filename in enumerate(filenames)],
        example_process_fn=updateAnnotated
    )     
    return indf


annotations = annotateYouTubeAudio(df, labels=labels)


## Some examples of A-to-B relationships
#### **A** (Reference Work)
- [Metallica: Nothing Else Matters (Official Music Video)](https://www.youtube.com/watch?v=tAGnKpE4NCI)

#### b_matches_a
- [Metallica - Nothing else matter lyrics](https://www.youtube.com/watch?v=x7bIbVlIqEc)

#### b_version_a
- [Metallica - Nothing Else Matters Live at Rock am Ring 2014](https://www.youtube.com/watch?v=RwpzlpPwWfc)
- [Nothing Else Matters - Metallica - William Joseph feels the Rain](https://www.youtube.com/watch?v=GdU6snztM0A)
- [Metallica - Nothing Else Matters cover by Jadyn Rylee](https://www.youtube.com/watch?v=OLE9qXN8Sqo)

#### b_unrelated_a
- [Queen - Bohemian Rhapsody (Freddie Mercury).](https://www.youtube.com/watch?v=mPbclTnMLlc)
- [Metallica: Enter Sandman (Official Music Video)](https://www.youtube.com/watch?v=CD-E-LDc384)

#### b_nonmusic
- [Jimmy Kimmel’s Interview with Metallica](https://www.youtube.com/watch?v=8pGy-8s0QoU)
